<a href="https://colab.research.google.com/github/ArefMahjoubfar/ShiftSet/blob/V2.0/Shift_Setter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Shift Set Script**
    Code by <*Aref Mahjoubfar* >  <4/29/2024>

Points to consider:

- Some indivs have some day limitations to have a shift
- Some indivs should be tagged with someone else: first version will not considering the tags. because the priority is day limitations. then with regarding the tags the next version will be produced based on minimal changing on the previous version.

- Indivs have diffrent number of shifts
- Days have diffrent degrees of difficulty
- In a day, shifts are in diffrent categories: site


- [X] 1st: prevent an indiv to has 2 shifts in a day.
- [X] 2nd: assign specific number of shifts to specific indivs.
- [ ] 3rd: consider holidays. at first you shoud choose individuals for holidays, then remove those days from total days and add the day before and after those holidays to the limitation days of those specific individuals.
- [ ] 4th: consider tags.

In [249]:
# install libraries
!pip install pandas

In [557]:
# import libraries
import pandas as pd
import random
import datetime
from tqdm import tqdm

In [523]:
# Load CSV files
Individuals_df = pd.read_csv('/content/drive/MyDrive/PY_Prac/SHIFT_SETTER/INDIVS.csv', encoding='utf-8')
Shift_days_df = pd.read_csv('/content/drive/MyDrive/PY_Prac/SHIFT_SETTER/SHIFT_DAYS.csv', encoding='utf-8')

In [571]:
# Parameters
def Set_params():
  global n_indivs, n_sites, n_days, n_shifts, Indivs, n_Indiv_Shift, Sites, Shifts, Days, Shifts_no_indiv, Limits, Assigned_shift_no
  # Set your Parameters here:
  # --------------------------------------------------------------------------------------------------------------

  n_indivs = 22   # number of individuals to have shifts in the course.
  n_sites = 5     # number of sites in a day / number of individuals in a day to have shifts / number of shifts in a day.
  n_days = 90     # number of days in the course
  n_shifts= n_days* n_sites  # number of total shifts

  # --------------------------------------------------------------------------------------------------------------

  # Generating list of Indivs, Sites and Shifts
  Indivs=[indiv for indiv in range(n_indivs)]
  n_Indiv_Shift=[0 for i in range(len(Indivs))]  # e,g. 20 shifts for each individual.
  Sites = ['site'+str(site) for site in range(n_sites)]
  Shifts = [[j for j in range(n_sites)]for i in range(n_days)]
  Days = [Day for Day in range(len(Shifts))]

  # a list that has length equal to number of indivs with elements of number of shifts for individuals
  Shifts_no_indiv = [0 for i in range(len(Indivs))]
  # Generating an empty list of day limitations for each individual --> Individual: Day limitations dictionary
  Limits={}
  for indiv in Indivs:
    Limits[indiv]=[]

  # Place to add extra limitation days for a specific individual
  #   ...
  #   ...

  # locate the exact place of Days and Shifts in the dataframes
  Indiv_list = Individuals_df.iloc[:,0]
  Day_list = Shift_days_df.iloc[:,0]
  Assigned_shift_no = Individuals_df.iloc[:n_indivs,1:2]    # the second coulmn in the csv file should be the assigned number of shifts for each individual.
  # reaching the total shifts assigned to an individual by using .loc[index]. e,g.
  Assigned_shift_no= Assigned_shift_no['TOTAL SHIFTS'].tolist()
# we can call the individuals who have shift in a specific day by using its index Shifts[Day_index].e,g.:
Shifts[0]

[13, 14, 4, 0, 20]

In [569]:
# the function that counts the shift numbers
def Counter(The_chosen_one):
  Shifts_no_indiv[The_chosen_one]+=1

In [570]:
# Generating list of available indivs
def available_indivs():
  available_indivs_list=[]
  for indiv in Indivs:
    # print(f'Day:{[Day]}   Shift:{[Shift]}  indiv: {[indiv]}   {Assigned_shift_no[indiv]}   {Shifts_no_indiv[indiv]}   available_indivs_list:{available_indivs_list}')
    if Assigned_shift_no[indiv] > Shifts_no_indiv[indiv]:   # check if an indiv has reach the upper limit of shifts number.
      if Day in Limits[indiv]:
          pass
      else:
          available_indivs_list.append(indiv)
          # print(f'available_indivs_list---------------> {available_indivs_list}')
    else:
      # print('==========================================================================================================================')
      pass
  return available_indivs_list

In [561]:
# the function that update the day limitations for an individual based on their shifts
def Update_Limits(The_chosen_one, Day):
      Limits[The_chosen_one].append(Day)
      # Preventing 48 hours Shifts(two shifts together)
      Limits[The_chosen_one].append(Day-1)
      Limits[The_chosen_one].append(Day+1)

In [562]:
# the function that will set an indiv for a specific shift
def ShiftSet(Shift):
    The_chosen_one = available_indivs_list[random.randrange(len(available_indivs_list))]
    Shifts_of_day[Shift]= The_chosen_one
    Update_Limits(The_chosen_one, Day)
    Counter(The_chosen_one)


In [564]:
def StartSet():
  global Day, Shifts_of_day, available_indivs_list
  for Day,Shifts_of_day in enumerate(Shifts):
      # print(Day, Shifts_of_day)
      for Shift in Shifts_of_day:
        available_indivs_list = available_indivs()
        ShiftSet(Shift)

In [ ]:
The_best_rnd_seed_rate = 0
The_best_rnd_seed = None
The_best_set=0
for i in tqdm(range(10000)):
  random.seed(i)
  try:
    Set_params()
    StartSet()
  except:
    successful_rate = (sum(Shifts_no_indiv)/sum(Assigned_shift_no))*100
    if successful_rate>The_best_rnd_seed_rate:
      The_best_rnd_seed = i
      The_best_rnd_seed_rate = successful_rate
      The_best_set = sum((Shifts_no_indiv))
    print(f'random.seed({i}):     <---->    ({sum(Shifts_no_indiv)} of {sum(Assigned_shift_no)}) -----  {successful_rate} %    The best: random.seed({The_best_rnd_seed})   {The_best_rnd_seed_rate}  ({The_best_set} of {sum(Assigned_shift_no)})')
  else:
    print('Successful!')
    break

In [567]:
def Play_Set_Shifts(seed):
  try:
    random.seed(seed)
    Set_params()
    StartSet()
  except:
    print(f'Shifts have been set with successful rate: {The_best_rnd_seed_rate}.  The last {sum(Assigned_shift_no)-(The_best_set)} shift/s remained. Try to set it manually:)')

Play_Set_Shifts(The_best_rnd_seed)
Shifts_df=pd.DataFrame(Shifts)
Shifts_df

Shifts have been set with successful rate: 99.77777777777777.  The last 1 shift/s remained. Try to set it manually:)


,0,1,2,3,4
0,13,14,4,0,20
1,18,9,1,2,3
2,10,14,11,13,15
3,12,5,9,8,2
4,17,21,0,6,20
...,...,...,...,...,...
85,16,19,17,4,0
86,15,2,9,3,20
87,4,12,16,17,0
88,15,3,20,2,9


In [551]:
print(Shifts_no_indiv, sum(Shifts_no_indiv))
print(Assigned_shift_no)

[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20] 449
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
